In [1]:
# Load IsoplotR
library('IsoplotR')
sessionInfo()

R version 4.1.3 (2022-03-10)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 22621)

Matrix products: default

locale:
[1] LC_COLLATE=English_United States.1252 
[2] LC_CTYPE=English_United States.1252   
[3] LC_MONETARY=English_United States.1252
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.1252    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] IsoplotR_5.2

loaded via a namespace (and not attached):
 [1] fansi_1.0.4     crayon_1.5.2    digest_0.6.31   utf8_1.2.3     
 [5] IRdisplay_1.1   repr_1.1.6      lifecycle_1.0.3 jsonlite_1.8.4 
 [9] evaluate_0.20   pillar_1.9.0    rlang_1.1.0     cli_3.6.1      
[13] uuid_1.1-0      vctrs_0.6.1     IRkernel_1.3.2  tools_4.1.3    
[17] glue_1.6.2      fastmap_1.1.1   compiler_4.1.3  base64enc_0.1-3
[21] pbdZMQ_0.3-9    htmltools_0.5.5

In [2]:
# Get files from isoplot directory
filenames = dir('isoplotr_raw',pattern='*.csv')
print(filenames)

 [1] "100211-1A.csv"   "100211-9A.csv"   "100411-5.csv"    "AB0913.csv"     
 [5] "AB0926.csv"      "CT130918-2A.csv" "CT130918-9A.csv" "CT130919-5A.csv"
 [9] "CT130919-8A.csv" "CT15076.csv"     "CT15082.csv"     "CT15092.csv"    
[13] "CT15099.csv"     "CT15113.csv"     "CT15127.csv"    


In [3]:
# Create new directories for concordia and MLA ages
dir.create(file.path('concordia'),showWarnings=FALSE)
dir.create(file.path('MLA'),showWarnings=FALSE)

# Calculate concordia and MLA ages for each sample and output full table
for (f in filenames) {
    
    # Calculate ages from isotopic data
    filepath <- file.path('isoplotr_raw',f)
    UPb <- read.data(filepath,method='U-Pb',format=1)
    
    # Ensure output ages have error at 2s and that concordia distance calculated
    tr <- age(UPb,oerr=2,discordance=discfilter('c'))
    
    # Output the age results to csv
    out_f <- sub('\\.csv$','_processed.csv',f)
    out_path <- file.path('concordia',out_f)
    write.csv(tr,file=out_path)
    
    # Do the discordance filter equivalent to 20% and -10% values from AgeCalcML
    df <- discfilter(option='c',cutoff=c(-3.5,9.4))
    
    # Make and save radial plot
    plot_name <- sub('\\.csv$','.jpg',f)
    plot_path <- file.path('MLA',plot_name)
    
    jpeg(plot_path,units='in',width=6,height=6,res=300)
    radialplot(UPb,k='min',cutoff.disc=df)
    dev.off()
    
    # Get minimum age from radial plot in separate table
    mla <- peakfit(UPb,k='min',cutoff.disc=df)
    mla_path <- file.path('MLA',sub('\\.csv$','_mla.csv',f))
    write.csv(mla$peaks,file=mla_path)
}